In [1]:
import os
import subprocess
import pandas as pd
import nhs_number
import json

In [2]:
path_synthea = "../../../synthea"
path_csv = path_synthea + "/output/csv"
path_patients = path_csv + "/patients.csv"
path_encounters = path_csv + "/encounters.csv"
path_output = "../../data/synthea.json"

In [3]:
cols_patients = ["Id", "BIRTHDATE", "FIRST", "LAST"]
cols_encounters = ["PATIENT", "ENCOUNTERCLASS", "REASONDESCRIPTION"]
cols_join = ["BIRTHDATE", "FIRST", "LAST", "REASONDESCRIPTION"]
cols = {
    "NHS_NUMBER": "NHS_NUMBER",
    "BIRTHDATE": "DATE_OF_BIRTH",
    "FIRST": "GIVEN_NAME",
    "LAST": "FAMILY_NAME",
    "REASONDESCRIPTION": "DIAGNOSIS",
}

In [4]:
cwd = os.getcwd()
os.chdir(path_synthea)

subprocess.run(["./run_synthea", "-p", "10", "West Yorkshire"])

os.chdir(cwd)

> Task :versionTxt
> Task :compileJava UP-TO-DATE
> Task :processResources UP-TO-DATE
> Task :classes UP-TO-DATE

> Task :run
Scanned 83 modules and 140 submodules.
Loading submodule modules/hiv/art_sequence_2015.json
Loading submodule modules/heart/acs_arrival_medications.json
Loading submodule modules/breast_cancer/tnm_diagnosis.json
Loading submodule modules/hiv/art_sequence_1997_2002.json
Loading submodule modules/heart/cabg/preoperative.json
Loading submodule modules/hiv/art_sequence_2006_2014.json
Loading submodule modules/covid19/nonsurvivor_lab_values.json
Loading submodule modules/covid19/survivor_lab_values.json
Loading submodule modules/covid19/diagnose_blood_clot.json
Loading submodule modules/dermatitis/moderate_cd_obs.json
Loading submodule modules/contraceptives/female_sterilization.json
Loading submodule modules/heart/chf_meds.json
Loading submodule modules/hiv/art_sequence_1987_1994.json
Loading submodule modules/covid19/supplies_hospitalization.json
Loading submodule 

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.


Loading submodule modules/dermatitis/mid_moderate_eczema_obs.json
Loading submodule modules/heart/stemi_fibrinolytic.json
Loading submodule modules/contraceptives/clear_contraceptive.json
Loading submodule modules/covid19/end_outcomes.json
Loading submodule modules/heart/chf_meds_hfmef.json
Loading submodule modules/heart/nsteacs_pathway.json
Loading submodule modules/surgery/general_anesthesia.json
Loading submodule modules/heart/avrr/preoperative.json
Loading submodule modules/veterans/veteran_suicide_probabilities.json
Loading submodule modules/medications/moderate_opioid_pain_reliever.json
Loading submodule modules/contraceptives/oral_contraceptive.json
Loading submodule modules/covid19/measurements_vitals.json
Loading submodule modules/medications/otc_antihistamine.json
Loading submodule modules/covid19/determine_risk.json
Loading Lookup Table: covid-19-severity-outcomes.csv
Loading Lookup Table: covid-19-survival-outcomes.csv
Loading submodule modules/covid19/admission.json
Loadi

In [5]:
df_patients = pd.read_csv(path_patients)[cols_patients]

df_encounters = pd.read_csv(path_encounters)[cols_encounters]
df_encounters = df_encounters[
    (df_encounters["ENCOUNTERCLASS"] != "wellness")
    & (df_encounters["REASONDESCRIPTION"].notna())
].drop_duplicates(subset="PATIENT")

df_join = df_patients.join(df_encounters.set_index("PATIENT"), on="Id")[
    cols_join
]

In [6]:
nhs_numbers = nhs_number.generate(quantity=len(df_join))

In [7]:
df_result = df_join.assign(NHS_NUMBER=nhs_numbers).rename(columns=cols)[
    cols.values()
]

In [8]:
df_result

,NHS_NUMBER,DATE_OF_BIRTH,GIVEN_NAME,FAMILY_NAME,DIAGNOSIS
0,6288322873,2013-02-08,Rachael,Ankunding,Otitis media
1,5356943318,2019-08-05,Gino,Sawayn,Acute bronchitis (disorder)
2,3717400924,2004-06-22,Jetta,Greenholt,Fracture of clavicle
3,9181374534,1996-08-01,Malik,Mante,Streptococcal sore throat (disorder)
4,7269014760,1982-07-22,Shaun,Zulauf,Normal pregnancy
5,1427245096,1973-05-10,Grace,Russel,Acute bronchitis (disorder)
6,7846469215,1968-03-05,Robt,Osinski,Sinusitis (disorder)
7,1823381790,1950-03-07,Marlana,Boehm,Perennial allergic rhinitis with seasonal vari...
8,8348963847,1966-03-19,Jesenia,Ritchie,Normal pregnancy
9,5805419122,1952-06-05,Seth,O'Kon,Acute viral pharyngitis (disorder)


In [9]:
array = []

for i in range(len(df_result)):
    array.append(df_result.iloc[i].to_dict())

output = json.dumps(array)

In [10]:
os.makedirs(os.path.dirname(path_output), exist_ok=True)

with open(path_output, "w") as f:
    f.write(output)